<a href="https://colab.research.google.com/github/Aaron-David-HdM/Applied-Data-Analytics/blob/main/UseCase_1/Datenimport_und_Bereinigung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Zeigt Inhalte des gewählten Ordners**

In [ ]:
ls

sample_data/


**Verknüpfung zum GitHub-Repository**

In [ ]:
!git clone https://github.com/Aaron-David-HdM/Applied-Data-Analytics.git

Cloning into 'Applied-Data-Analytics'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 173 (delta 3), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (173/173), 44.84 KiB | 2.04 MiB/s, done.
Resolving deltas: 100% (60/60), done.


**Wechselt zum lokalen Repository-Verzeichnis und holt Daten vom GitHub-Repo**

In [ ]:
%cd /content/Applied-Data-Analytics/
!git pull

**Bibliotheken-Import, Wechsel zum lokalen Repository und Laden der CSV-Dateien**

In [ ]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype

%cd /content/Applied-Data-Analytics/UseCase_1

filepath_1 = 'Rohdaten_18-34_Jahre.csv' # Gesamtzahl der Nutzer,18-24 sowie 25-34 jährige Nutzer
filepath_2 = 'Rohdaten_35-65+_Jahre.csv' # 35-44,	45-54 und 55-64 jährige Nutzer sowie Nutzer über 65 Jahren


df_1 = pd.read_csv(filepath_1, decimal=',')
df_2 = pd.read_csv(filepath_2, decimal=',')

/content/Applied-Data-Analytics/UseCase_1


**Daten Bereinigung**

Google Analytics erlaubt nur die Verknüpfung von bis zu 4 verschiedene Segmenten bei der Erzeugung der Daten. Somit waren wir gezwungen, zwei Tabellen anzulegen und diese im Anschluss miteinander zu 'mergen'.


**Löschen von NaN-Werten und der 'Zeitraum'-Column sowie Ersetzen der Tausendertrennzeichen bei Zahlenwerten**

In [ ]:
new_df_1 = df_1.dropna()
new_df_1 = df_1.drop(columns=['Zeitraum'])
new_df_1['Nutzer'] = new_df_1['Nutzer'].str.replace('.', '')

new_df_2 = df_2.dropna()
new_df_2 = df_2.drop(columns=['Zeitraum'])
new_df_2['Nutzer'] = new_df_2['Nutzer'].str.replace('.', '')

**Columns umbenannt und neu gruppiert**

In [ ]:
new_df_1 = new_df_1.rename(columns={'Segment':'Altersgruppe', 'Index: Tag':'Tag'})
new_df_1 = new_df_1.groupby(['Tag', 'Altersgruppe'])['Nutzer'].first().unstack()
new_df_1.columns = ['Alle Nutzer', '18-24', '25-34']

new_df_2 = new_df_2.rename(columns={'Segment':'Altersgruppe', 'Index: Tag':'Tag'})
new_df_2 = new_df_2.groupby(['Tag', 'Altersgruppe'])['Nutzer'].first().unstack()
new_df_2.columns = ['35-44', '45-54', '55-64', '65+']

**Nutzerzahlen zu Integern konvertiert**

In [ ]:
new_df_1[['18-24', '25-34', 'Alle Nutzer']] = new_df_1[['18-24', '25-34', 'Alle Nutzer']].apply(np.int64)

new_df_2[['35-44', '45-54', '55-64', '65+']] = new_df_2[['35-44', '45-54', '55-64', '65+']].apply(np.int64)

**Die zwei Dataframes zu einem Dataframe zusammengeführt**

In [ ]:
df = new_df_1.merge(new_df_2, how='inner', left_index=True, right_index=True)

**Löschen der Column 'Alle Nutzer', da sie für uns irrelevante Daten enthält**

In [ ]:
df.drop(columns=['Alle Nutzer'])

,18-24,25-34,35-44,45-54,55-64,65+
Tag,,,,,,
01.08.21,144,177,96,50,45,23
02.08.21,253,454,211,142,74,50
03.08.21,279,397,201,127,69,61
04.08.21,306,407,225,120,77,37
05.08.21,243,392,232,140,66,52
06.08.21,214,338,162,77,54,37
07.08.21,130,139,84,38,44,22
08.08.21,146,196,90,51,22,23
09.08.21,256,387,186,131,46,58


**Nutzerzahlen je Tag addieren und Werte zum Datenframe hinzufügen**

In [ ]:
df['Alle Nutzer'] = df.iloc[:, 1:].sum(axis=1)

**Dataframe als CSV-Datei abspeichern**

In [ ]:
df.to_csv('Werbung_A.csv', sep=',')